In [1]:
import bokeh.plotting as bk
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.append('../') # go to parent dir
import numpy as np
np.set_printoptions(precision=3, formatter={'float': '{: 0.3f}'.format})
np.seterr('warn')
import os
import cProfile, pstats, io

In [3]:
from bokeh.palettes import Category20
colors = Category20[20]

## Spline Data Generation

### Create Config

In [4]:
steps = 100
dt = 0.04

config = {
    'steps': steps + 1,
    'd': 2,
    'sd': 5,
    'r': 0.05 ** 2 * np.identity(2),
    'init_m': np.asarray([6.5, 2.5, 0.00, 12, 0.1]),
    'p_basis': np.array([
        [2.5, 0.0],
        [2.5, 1.0],
        [0.0, 1.0],
        [-2.5, 1.0],
        [-2.5, 0.0],
        [-2.5, -1.0],
        [0.0, -1.0],
        [2.5, -1.0],
    ]),
    'mean_scat_number': 125,
}

### Run Simulator

In [5]:
from data import SplineDataSimulator as Simulator

data_source = Simulator(dt=dt, **config)

for i in range(steps):
    data_source.step()

gt, measurements = data_source.extract()
bboxes = data_source.extrackt_bbox()

In [6]:
from bokeh.models import ColumnDataSource

meas_source = ColumnDataSource(
        data={
            'ts': measurements['ts'],
            'x': measurements['xy'][:, 0],
            'y': measurements['xy'][:, 1],
        }
)

gt_bbox_source = ColumnDataSource(
        data={
            'ts': bboxes['ts'],
            'm_x': bboxes['center_xy'][:, 0],
            'm_y': bboxes['center_xy'][:, 1],
            'phi': bboxes['orientation'][:],
            'l': bboxes['dimension'][:, 0],
            'w': bboxes['dimension'][:, 1],
        }
)

gt_source = ColumnDataSource(
        data={
            'ts': gt['ts'],
            'x': gt['m'][:, 0],
            'y': gt['m'][:, 1],
            'phi': gt['m'][:, 2],
            'omega': gt['m'][:, 3],
            'v': gt['m'][:, 3],
        }
)

In [7]:
from bokeh.layouts import column
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, BoxSelectTool, PanTool, BoxZoomTool, WheelZoomTool, UndoTool, RedoTool, ResetTool, SaveTool

TOOLS = [PanTool(), BoxSelectTool(), BoxZoomTool(), WheelZoomTool(), UndoTool(), RedoTool(), ResetTool(), SaveTool()]

top = figure(tools=TOOLS, width=800, height=350, title=None, match_aspect=True, output_backend="webgl")
meas_scatter = top.circle('x', 'y', color='#303030', legend='measurements', size=1, alpha=0.2, source=meas_source)
track_plot = top.line('x', 'y', legend='track', source=gt_source)
hover = HoverTool(renderers=[track_plot],
        tooltips=[
            ("index", "$index"),
            (r"(x, y, phi, v)", "(@x, @y, @phi, @v)"),
        ]
    )
top.add_tools(hover)

bottom = figure(tools=TOOLS, width=800, height=350, title=None, 
                x_range=top.x_range, y_range=top.y_range, output_backend="webgl")
bottom.circle('x', 'y', color='#303030', legend='measurements', size=1, alpha=0.2, source=meas_source)
bottom.rect(x='m_x', y='m_y', width='l', height='w', angle='phi', color='#98df8a', 
            fill_alpha=0.2, source=gt_bbox_source, legend='ground truth boxes')

bottom.legend.click_policy="hide"

p = column([top, bottom])
show(p)

### Save Data

In [16]:
path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(path):
    os.makedirs(path)

filename = r'simulated_data'
np.save(os.path.join(path, filename + '.npy'), measurements)
filename = r'gt_path'
np.save(os.path.join(path, filename + '.npy'), gt)
filename = r'gt_bboxes'
np.save(os.path.join(path, filename + '.npy'), bboxes)